In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/test_tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf

# 1. 환경 설정 및 데이터 불러오기


In [7]:
%%bash
mkdir ./submission

In [8]:
!apt install pigz pv > /dev/null

In [9]:
import nltk
from nltk.corpus import brown
from nltk import pos_tag
import nltk.corpus

# 명사만 가져옴
words = nltk.corpus.words
tagged_words = pos_tag(words.words())
nouns = [word for word, pos in tagged_words if pos in ('NN', 'NNS')]

In [14]:
tagged_words[:5]

[('A', 'DT'), ('a', 'DT'), ('aa', 'JJ'), ('aal', 'NN'), ('aalii', 'NN')]

In [15]:
# 대충 명사만 잘 가져왔겠지 싶다.
nouns[:5]

['aal', 'aalii', 'aam', 'aardvark', 'aardwolf']

In [16]:
# 10만 개의 단어를 전부 써서 하기엔 메모리에 올라가기 버거울 것 같다.
len(nouns)

108748

In [17]:
torch.cuda.get_device_name()

'Tesla T4'

In [18]:
import torch
GPU_NUM = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(0)

### Bert는 너무 느려서 못쓰겠다...

In [19]:
# 클러스터링을 통해 일부만 사용해보자.
from transformers import BertModel, BertTokenizer

# BERT 모델과 토크나이저
model = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model)
model = BertModel.from_pretrained(model)


model.eval()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def get_bert_embedding(word, tokenizer, model):
    inputs = tokenizer(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

embeddings = [get_bert_embedding(word, tokenizer, model) for word in nouns]

## 대신 Word2Vec을 써보자

In [5]:
from gensim.models import KeyedVectors

model_path = '/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [20]:
words_with_embeddings = []
embeddings = []

for word in nouns:
    if word in word2vec:
        words_with_embeddings.append(word)
        embeddings.append(word2vec[word])

In [24]:
# 모든 명사들 중 google news에 나오는 '일상에서 흔히 보고 듣는 단어'들은 대충 이정도인가 보다.
len(embeddings), len(words_with_embeddings)

(27327, 27327)

In [25]:
words_with_embeddings[:5]

['aal', 'aam', 'aardvark', 'abacus', 'abaft']

In [26]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

# K-Means 클러스터링을 통해 2000개를 샘플링하려고 한다. // 시간이 느려서 기각.
num_clusters = 2000
minibatch_kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=42, batch_size=1000)
minibatch_kmeans.fit(embeddings)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(batch_size=1000, n_clusters=2000, random_state=42)

In [27]:
# 각 클러스터의 중심에 가장 가까운 단어 선택
closest_words = []

for i in range(num_clusters):
    cluster_center = minibatch_kmeans.cluster_centers_[i]
    distances = np.linalg.norm(embeddings - cluster_center, axis=1)
    closest_word_idx = np.argmin(distances)
    closest_words.append(words_with_embeddings[closest_word_idx])

In [41]:
def is_concrete_noun(noun):
    # 개념적 명사를 나타낼 수 있는 접미사 리스트
    abstract_suffixes = ["ness", "ity", "ism", "tion", "ment", "ance", "ence", "ship", "ability", "al", "ence", "dom", "hood", "less"]

    # 만약 단어가 개념적 접미사를 포함하면 False 반환
    return not any(noun.endswith(suffix) for suffix in abstract_suffixes)

In [42]:
# 선택된 2000개의 샘플
closest_words[:20]

['effectivity',
 'vealer',
 'cornucopia',
 'gingerroot',
 'headquarter',
 'convenient',
 'sulfa',
 'concupiscent',
 'anime',
 'echoic',
 'kotwal',
 'agreeability',
 'banger',
 'telescope',
 'hala',
 'kinescope',
 'autogiro',
 'boon',
 'charade',
 'thoracentesis']

In [43]:
filtered_nouns = [noun for noun in closest_words if is_concrete_noun(noun)]

In [44]:
# 개념적인 단어들은 싹 다 정리했음.
len(filtered_nouns)

1669

In [45]:
filtered_nouns[:20]

['vealer',
 'cornucopia',
 'gingerroot',
 'headquarter',
 'convenient',
 'sulfa',
 'concupiscent',
 'anime',
 'echoic',
 'banger',
 'telescope',
 'hala',
 'kinescope',
 'autogiro',
 'boon',
 'charade',
 'thoracentesis',
 'release',
 'bromine',
 'swank']

In [ ]:
"""
   여기까지가 준비단계였음.
""" 




In [46]:
# import Tokenizer and Model
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
import os
import sys
import shutil

# 에이전트 패스 설정
KAGGLE_AGENT_PATH = './kaggle_simulations/agent/'

if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, '1')
else:
    model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

# 모델 및 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, device_map="auto")
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [47]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/kaggle/input/llama-3/transformers/8b-chat-hf/1', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	12

In [48]:
# Class Observation
class Observation:
    def __init__(self):
        self.step = 0
        self.mode = 'answering'
        self.role = 'answerer'
        self.turnType = 'answer'
        self.keyword = 'power cable'
        self.category = 'things'
        self.questions = ['is it a place?']
        self.answers = []
        self.guesses = []
        
obs = Observation()

In [13]:
# 원본에서 가져온 generate_answer
def generate_answer(template):
    inp_ids = tokenizer(template, return_tensors="pt").to("cuda")
    out_ids = model.generate(**inp_ids,max_new_tokens=15).squeeze()
    start_gen = inp_ids.input_ids.shape[1]
    out_ids = out_ids[start_gen:]
    if id_eot in out_ids:
        stop = out_ids.tolist().index(id_eot)
        out = tokenizer.decode(out_ids[:stop])
    else:
        out = tokenizer.decode(out_ids)
    return out
    
# 수정된 answerer
def answerer(obs):
    sys_prompt = f"""you are a helpful AI assistant, and your are very smart in judging correctness.
    the role of you is answering to question. 
    you must say 'yes' or 'no', any other answer is invalid.
    to help you, here's an example of how it should work assuming that the 'obs.keyword' is "Morocco":
    examle: 
    <user: is it a place?
    you: "yes" >"""

		# bos와 함께 system prompt 전달.
    chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{sys_prompt}<|eot_id|>"""
    chat_template += "<|start_header_id|>user<|end_header_id|>\n\n"
    
    # user message
    chat_template += f"'keyword is'{obs.keyword}, {obs.questions[0]}'<|eot_id|>"
    
    # assistant token
    chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
    
    # 모델에서 실제로 돌아가는 부분.
    output = generate_answer(chat_template)
    return output

In [9]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [16]:
answer = answerer(obs=obs)
answer

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2024-08-02 14:08:04.558147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 14:08:04.558258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 14:08:04.684983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'no'

# Information Gain
- get answers('yes or no') to all the open keywords

## 1) Get Open Keywords

- DataFrame형식으로 모든 오픈키워드 저장

In [12]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
import json
import pandas as pd
from keywords_local import KEYWORDS_JSON

def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)

    keyword_list = []
    category_list = []
    alts_list = []

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])

    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd
    
keywords_df = create_keyword_df(KEYWORDS_JSON)

--2024-08-02 15:56:17--  https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93552 (91K) [text/plain]
Saving to: 'keywords_local.py'

keywords_local.py   100%[===================>]  91.36K  --.-KB/s    in 0.008s  

2024-08-02 15:56:17 (10.9 MB/s) - 'keywords_local.py' saved [93552/93552]



In [13]:
print(f'total_length = {keywords_df.shape[0]}')
keywords_df.head()

total_length = 1142


keyword category                alts
0    Advertisement   things                  []
1            Agave   things                  []
2   Air compressor   things                  []
3  Air Conditioner   things  [Air Conditioning]
4       Air filter   things                  []

In [26]:
answer_list = []

keywords = keywords_df['keyword']
for keyword in keywords:
    obs.keyword = keyword
    answer = answerer(obs)
    answer_list.append(answer)

keywords_df[obs.questions[0]] = answer_list

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [43]:
keywords_df[obs.questions[0]] = answer_list
keywords_df

keyword category                   alts is it a place?
0       Advertisement   things                     []             no
1               Agave   things                     []             no
2      Air compressor   things                     []             no
3     Air Conditioner   things     [Air Conditioning]             no
4          Air filter   things                     []             no
...               ...      ...                    ...            ...
1137            rhine    place                     []            yes
1138    yangtze river    place  [changjiang, yangtze]            yes
1139     yellow river    place             [huang he]            yes
1140    zambezi river    place              [zambezi]            yes
1141    yenisei river    place              [yenisei]            yes

[1142 rows x 4 columns]

In [47]:
keywords_df[(keywords_df['category']=='things') & (keywords_df['is it a place?']=='yes')]

keyword category          alts is it a place?
12            Aquarium   things            []            yes
14          Auditorium   things            []            yes
19                Bank   things            []            yes
22                 Bat   things            []            yes
24        Battery Bank   things            []            yes
28           Bike path   things  [Bike trail]            yes
32           Bleachers   things            []            yes
43          Break room   things            []            yes
84         Coral reefs   things            []            yes
124              Earth   things            []            yes
158       Fishing pier   things        [pier]            yes
189                GPS   things            []            yes
199        Guard tower   things            []            yes
228              Holly   things            []            yes
259              Kiosk   things            []            yes
284            Matches   things            []            yes
290   Medicine Cabinet   things            []            yes
313           Name tag   things            []            yes
335                Pad   things            []            yes
357             Plaque   things            []            yes
360              Plate   things            []            yes
374          Press Box   things            []            yes
379            Pry Bar   things            []            yes
385            Pyramid   things            []            yes
397     Rehearsal room   things            []            yes
403        Restaurants   things            []            yes
415              Sauna   things            []            yes
459            Stadium   things            []            yes
466         Steam room   things            []            yes
469        Storage bin   things            []            yes
471        Street sign   things            []            yes
486       Tennis court   things            []            yes
507       Tractor shed   things            []            yes
516             Turkey   things            []            yes
523  Underground River   things            []            yes
544            Volcano   things            []            yes
550        Wall Street   things            []            yes
553              Water   things            []            yes
563        Willow Tree   things      [Willow]            yes

In [48]:
keywords_df[(keywords_df['category']=='place') & (keywords_df['is it a place?']=='no')]

keyword category alts is it a place?
1114        k2    place   []             no
1117   cho oyu    place   []             no
1129  sumantri    place   []             no

In [14]:
obs.questions = ['does it have longitude and latitude?']

In [15]:
answer_list = []

keywords = keywords_df['keyword']
for keyword in keywords:
    obs.keyword = keyword
    answer = answerer(obs)
    answer_list.append(answer)

keywords_df[obs.questions[0]] = answer_list

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2024-08-02 15:56:20.315015: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 15:56:20.315115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 15:56:20.441802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
keywords_df

keyword category                   alts  \
0       Advertisement   things                     []   
1               Agave   things                     []   
2      Air compressor   things                     []   
3     Air Conditioner   things     [Air Conditioning]   
4          Air filter   things                     []   
...               ...      ...                    ...   
1137            rhine    place                     []   
1138    yangtze river    place  [changjiang, yangtze]   
1139     yellow river    place             [huang he]   
1140    zambezi river    place              [zambezi]   
1141    yenisei river    place              [yenisei]   

     does it have longitude and latitude?  
0                                      no  
1                                     yes  
2                                      no  
3                                      no  
4                                      no  
...                                   ...  
1137                                  yes  
1138                                  yes  
1139                                  yes  
1140                                  yes  
1141                                  yes  

[1142 rows x 4 columns]

In [18]:
keywords_df[(keywords_df['category']=='things') & (keywords_df['does it have longitude and latitude?']=='yes')]

keyword category      alts does it have longitude and latitude?
1              Agave   things        []                                  yes
7            Analogy   things        []                                  yes
8            Anemone   things        []                                  yes
10         Apple pie   things        []                                  yes
12          Aquarium   things        []                                  yes
..               ...      ...       ...                                  ...
560           Webcam   things        []                                  yes
561  Wheelchair ramp   things        []                                  yes
563      Willow Tree   things  [Willow]                                  yes
577           Yogurt   things        []                                  yes
578             Zinc   things        []                                  yes

[142 rows x 4 columns]

In [20]:
keywords_df[(keywords_df['category']=='place') & (keywords_df['does it have longitude and latitude?']=='no')]

Empty DataFrame
Columns: [keyword, category, alts, does it have longitude and latitude?]
Index: []

In [ ]:
def generate_answer(template):
    inp_ids = tokenizer(template, return_tensors="pt").to("cuda")
    out_ids = model.generate(**inp_ids,max_new_tokens=15).squeeze()
    start_gen = inp_ids.input_ids.shape[1]
    out_ids = out_ids[start_gen:]
    
    # Modified, but acting has same flow
    if id_eot in out_ids:
        stop = out_ids.tolist().index(id_eot)
        out = tokenizer.decode(out_ids[:stop])
    else:
        out = tokenizer.decode(out_ids)
    return out

In [37]:
def ask_question(question):
    # 질문에 대해 모델이 답변이 생성
    inputs = tokenizer(question, return_tensors='pt').to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=10)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip().lower()

def answerer(obs):
    sys_prompt = f"""you are a helpful AI assistant, and your are very smart in judging correctness.
    the role of you is answering to question.
    you must say 'yes' or 'no', any other answer is invalid.

    to help you, here's an example of how it should work assuming that the keyword is Morocco in the category "place":
    examle:
    <question: is it a place?
    you: "yes or no" >"""

#     chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{sys_prompt}<|eot_id|>"""
    chat_template = f"""{tokenizer.bos_token}  """
    
    return chat_template



In [ ]:
answerer_template

In [8]:
temp_df = keywords_df.iloc[3]
temp_df['keyword'] = [temp_df['keyword']] + temp_df['alts']
temp_df

/tmp/ipykernel_34/1755209794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['keyword'] = [temp_df['keyword']] + temp_df['alts']


keyword     [Air Conditioner, Air Conditioning]
category                                  place
alts                         [Air Conditioning]
Name: 3, dtype: object

In [23]:
len(keywords_df)

1142

In [7]:
keywords_df

keyword category                   alts
0       Advertisement    place                     []
1               Agave    place                     []
2      Air compressor    place                     []
3     Air Conditioner    place     [Air Conditioning]
4          Air filter    place                     []
...               ...      ...                    ...
1137            rhine    place                     []
1138    yangtze river    place  [changjiang, yangtze]
1139     yellow river    place             [huang he]
1140    zambezi river    place              [zambezi]
1141    yenisei river    place              [yenisei]

[1142 rows x 3 columns]

1. Information Gain
주어진 키워드를 기반으로 최대한 많은 정보를 얻을 수 있는 질문을 하기
> 사람들이 질문했던 것들을 가져와서 어떻게 나뉘어지나 보면 어떨까?
>> 그러면 일단 다른 사람들이 했던 질문들을 가져와야겠넹 ㅎㅎㅎㅎㅎ...크롤링?


In [32]:
%%writefile -a submission/main.py

!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
from keywords_local import KEYWORDS_JSON
import json

# import Tokenizer and Model
from transformers import AutoTokenizer, AutoModelForCasualLM
import torch
import os
import sys
import shutil

# 에이전트 패스 설정
KAGGLE_AGENT_PATH = './kaggle_simulations/agent/'

if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, '1')
else:
    model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

# 모델 및 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCasualLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, device_map="auto")
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]

# open keywords
json_data = json.loads(KEYWORDS_JSON)
things = json_data[0]['words']
place = json_data[1]['words']
keywords = things + place


 


class Robot:
    def __init__(self, keywords):
        self.keywords = keywords
    
    def generate_questions(self, keywords, num_questions=5):
        questions = []
        
        # 첫 번째 질문 고정
        questions.apend("Is it a place")
        
        # 두 번째 질문은 첫 번째 질문의 답변에 따라 결정
        if self.history and self.history[-1]['answer'] == 'no':
            questions.append("Is it a thing related to food or dring in anyway?")
        else:
            questions.append("does the keyword relate to a country?")
    
        if self.history and self.history[-1]['answer'] == 'yes':
            # Place와 관련된 질문
            for _ in range(num_questions):
                question = random.choice(place_questions)
                questions.append(question)
                place_question.remove(question) # 중복 방지
        else:
            # Thing과 관련된 질문
            for _ in range(num_questions):
                category = random.choice(broad_categories)
                question = f"Would the keyword be included in the broad category of {category}?"
                questions.append(question)
                broad_categories.remove(category) # 중복 방지
                            
        # keyword가 30개 미만인 경우, 가장 많은 스펠링부터 소거법으로 진행.
        if len(keywords) <= 30:
            for first_letter in 'abcdefghijklmnopqrstuvwxyz':
                keywords_dict={}
                questions.append(f"Does the keyword begin with the letter {first_letter}?")
                
        return questions
    
    
    # yes와 no의 응답률 차이가 가장 적게 나는 질문을 사용하기 위함.
    def calculate_difference(self, question, keywords):        
        yes_keywords = self.filter_keywords(keywords, question, "yes")
        no_keywords = self.filter_keywords(keywords, question, "no")
        
        difference = abs(len(yes_keywords) - len(no_keywords))
        return difference
        
    def filter_keywords(self, keywords, question, answer):
        if answer == "yes":
            filtered_keywords = [kw for kw in keywords if question in kw]
        else:
            filtered_keywords = [kw for kw in keywords if question not in kw]
        return filtered_keywords
    
    def ask_question(self, question):
        # 질문에 대해 모델이 답변이 생성
        inputs = tokenizer(question, return_tensors='pt').to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=10)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return answer.strip().lower()
    
    def update_history_and_keywords(self, question, answer):
        # 히스토리 업데이트
        self.history.append({'question': question, 'answer': answer})
        
        # 키워드 필터링
        if answer == 'no':
#             self.keywords = 1234
            pass
        else:
            pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            #launch the asker role
            output = self.asker(obs)
        if mode == "answering":
            #launch the answerer role
            output = self.answerer(obs)
            if "yes" in output.lower():
                output = "yes"
            elif "no" in output.lower():
                output = "no"   
            if ("yes" not in output.lower() and "no" not in output.lower()):
                output = "yes"
        if mode == "guessing":
            #launch the guesser role
            output = self.asker(obs)
        return output    
    
    def asker(self, obs):
        sys_prompt = """You are a helpful AI assistant, and your are very smart in playing 20 questions game,
        the user is going to think of a word, it can be only one of the following 3 categories:
        1. a place
        2. a thing
        So focus your area of search on these options. and give smart questions that narrows down the search space\n"""
    
        if obs.turnType =="ask":
            if obs.step == 1:
                out
            ask_prompt = sys_prompt + """your role is to find the word by asking him up to 20 questions, your questions to be valid must have only a 'yes' or 'no' answer.
            to help you, here's an example of how it should work assuming that the keyword is Morocco:
            examle:
            <you: is it a place?
            user: yes
            you: is it in europe?
            user: no
            you: is it in africa?
            user: yes
            you: do most people living there have dark skin?
            user: no
            user: is it a country name starting by m ?
            you: yes
            you: is it Morocco?
            user: yes.>

            the user has chosen the word, ask your first question!
            please be short and not verbose, give only one question, no extra word!"""
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{ask_prompt}<|eot_id|>"""
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
            if len(obs.questions)>=1:
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                    chat_template += f"{a}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""Question: {q}\nAnswer: {a}\n"""
            guess_prompt =  sys_prompt + f"""so far, the current state of the game is as following:\n{conv}
            based on the conversation, can you guess the word, please give only the word, no verbosity around"""
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{guess_prompt}<|eot_id|>"""
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
                
        output = generate_answer(chat_template)        
        return output
        
        
        
    def answerer(self, obs):
        sys_prompt = f"""you are a helpful AI assistant, and your are very smart in playing 20 questions game,
        the role of the user is to guess the word by asking you up to 20 questions, your answers to be valid must be a 'yes' or 'no', any other answer is invalid and you lose the game.
        Know that the user will always guess a word belonging to one of the following 2 categories:
        1. a place
        2. a thing
        so make sure you understand the user's question and you understand the keyword you're playig on.
        for now the word that the user should guess is: "{obs.keyword}", it is of category "{obs.category}",
        to help you, here's an example of how it should work assuming that the keyword is Morocco in the category "place":
        examle:
        <user: is it a place?
        you: yes
        user: is it in europe?
        you: no
        user: is it in africa?
        you: yes
        user: do most people living there have dark skin?
        you: no
        user: is it a country name starting by m ?
        you: yes
        user: is it Morocco?
        you: yes.>"""
        
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{sys_prompt}<|eot_id|>"""
        chat_template += "<|start_header_id|>user<|end_header_id|>\n\n"
        chat_template += f"{obs.questions[0]}<|eot_id|>"
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{a}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                chat_template += f"{q}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        output = generate_answer(chat_template)
        return output
    
    
robot = Robot()


def agent(obs, cfg):
    
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:
        response = "yes"
        
    return response

Writing submission/main.py


FileNotFoundError: [Errno 2] No such file or directory: 'submission/main.py'

In [12]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/input/llama-3/transformers/8b-chat-hf . -C /kaggle/working/submission .

tar: /kaggle/input/llama-3/transformers/8b-chat-hf: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


# 파일 살펴보기

데이터셋이 업데이트되었다.
kaggle-environments/kaggle_environments/envs/llm_20_questions
/keywords.py  
(https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/keywords.py)
- 확인해보면 country, city, landmark 가 전부 place로 바뀌었고,
- things가 대거 추가되었다고 한다.

In [7]:
# 기존 로컬 패스
# file_path = "kaggle_environments/envs/llm_20_questions/keywords.py" 

# data file in github
# 데이터 업데이트로 키워드에 있는 내용이 조금 수정됨.
# Hidden keywords가 존재하므로, decision tree base로 진행하기에는 조금 어려울 수 있을듯.
!wget https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/keywords.py -O keywords.py

--2024-07-05 15:31:39--  https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/keywords.py
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'keywords.py'

keywords.py             [  <=>               ] 392.88K  1.54MB/s    in 0.2s    

2024-07-05 15:31:40 (1.54 MB/s) - 'keywords.py' saved [402310]



아이디어  
1) vector db를 만드는 건 어떨까? (어떻게 만드는건진 아직 모름)

In [16]:
# 키워드 파일 경로 추가
import sys
import json
from pprint import pprint as pp

keywords_path = "/kaggle/input/llm-20-questions/llm_20_questions/"
sys.path.insert(0, keywords_path)
json_file_path = '/kaggle/input/llm-20-questions/llm_20_questions/llm_20_questions.json'


# 키워드 파일 임포트 시도
try:
    import keywords
    print("keywords 모듈 임포트 성공!")
except ModuleNotFoundError:
    print("keywords 모듈을 찾을 수 없습니다. 경로를 확인하세요.")


keywords 모듈 임포트 성공!


In [9]:
with open(file_path) as json_file:
    json_data = json.load(json_file)

keywords 모듈 임포트 성공!


In [19]:
file_path = "./keywords.py"

In [25]:
with open('./keywords.py', 'r') as file:
    content = file.read()
    print(content)







<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-efd2f2257c96.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-6b1e37da2254.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" medi

In [22]:
# Keywords가 어떻게 생긴 파일인지 조금 더 살펴보자 (EDA)

import json

# json file의 형태로 되어있어서, 이것만 골라내려 함.
with open(file_path, 'r') as f:
    file_contents = f.read()
    
    # JSON 데이터만 추출
    start_idx = file_contents.find('KEYWORDS_JSON = """') + len('KEYWORDS_JSON = """')
    end_idx = file_contents.find('"""', start_idx)
    json_data_str = file_contents[start_idx:end_idx].strip()
    
    # JSON parsing
#     json_data = json.loads(json_data_str)

In [27]:
import requests

# GitHub 파일 URL
url = 'https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/keywords.py'

# 파일 다운로드
response = requests.get(url)
response.raise_for_status()  # 요청에 실패하면 예외를 발생시킵니다.

# 파일 저장
with open('downloaded_data.py', 'w', encoding='utf-8') as file:
    file.write(response.text)

print("파일이 성공적으로 다운로드되었습니다.")

<_io.TextIOWrapper name='downloaded_data.py' mode='w' encoding='utf-8'>
파일이 성공적으로 다운로드되었습니다.


In [30]:
# 파일을 열고 데이터 부분을 추출합니다.
with open('downloaded_data.py', 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)
# # "KEYWORDS_JSON = """ 부분을 찾아서 JSON 데이터를 추출합니다.
# start = content.find('KEYWORDS_JSON = """') + len('KEYWORDS_JSON = """')
# end = content.find('"""', start)
# json_data = content[start:end].strip()

# print(json_data)  # JSON 데이터 확인







<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-efd2f2257c96.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-6b1e37da2254.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" medi

In [23]:
pp(json_data[0])

KeyError: 0

In [ ]:
# category 별 분포.
# 첫 질문이 is it a city? 로만 시작해도 절반은 거르고 갈 수 있음
categories = [(item['category'], len(item['words'])) for item in json_data]
print(categories)

In [ ]:
json_data[1]['words']